In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#IMPORT CLASSES (from current directory)
sys.path.append(codeDirectory)
from CLASSES_CalculateMoreVariables import ModelData_Class, DataManager_Class

In [5]:
####################################
#LOADING CLASSES

In [6]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str)

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133


In [7]:
####################################
#FUNCTIONS

In [8]:
#Calculation Functions
def GetCloudyUpdraftThresholds():
    w_thresh1=0.1
    w_thresh2=0.5
    qcqi_thresh=1e-6
    return w_thresh1,w_thresh2,qcqi_thresh

def GetInputVariables(inputDataDirectory, timeString):
    w_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="winterp")
    qc_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qc")
    qi_data = DataManager.GetTimestepData(inputDataDirectory, timeString, VariableName="qi")
    qcqi_data=qc_data+qi_data

    InputDictionary = {'w_data': w_data,
                       'qcqi_data': qcqi_data}
    return InputDictionary
    
def VariableCalculation(inputDictionary):
    [w_data, qcqi_data] = (inputDictionary[k] for k in ['w_data', 'qcqi_data'])
    [w_thresh1,w_thresh2,qcqi_thresh] = GetCloudyUpdraftThresholds()

    #calculating A_g binary array
    cond1 = w_data >= w_thresh1
    cond2 = qcqi_data < qcqi_thresh
    A_g = np.where(cond1 & cond2, True, False)

    #calculating A_c binary array
    cond1 = w_data >= w_thresh2
    cond2 = qcqi_data >= qcqi_thresh
    A_c = np.where(cond1 & cond2, True, False)

    outputDictionary = {
        'A_c': A_c,
        'A_g': A_g
    }
    return outputDictionary

In [13]:
# ============================================================
# DataManager_Class
# ============================================================

#Libraries
import os
import h5py

class DataManager_Class:
    def __init__(self, mainDirectory, scratchDirectory, res, t_res, Nz_str, dataName, dtype):
        self.mainDirectory = mainDirectory
        self.scratchDirectory = scratchDirectory
        self.res = res
        self.t_res = t_res
        self.Nz_str = Nz_str
        self.dataName = dataName
        self.dtype = dtype

        # Initialize directories on creation
        self.inputDirectory = self.GetInputDirectory(mainDirectory, scratchDirectory, res)
        self.outputDirectory = self.GetOutputDirectory(mainDirectory, scratchDirectory, res)
        self.inputDataDirectory = self.MakeInputDataDirectory(self.inputDirectory, res, t_res, Nz_str)
        self.outputDataDirectory = self.MakeOutputDataDirectory(self.outputDirectory, res, t_res, Nz_str)

    # ============================================================
    # ========== Functions ==========
    # ============================================================

    def GetInputDirectory(self, mainDirectory, scratchDirectory, res):
        if res == '1km':
            inputDirectory = os.path.join(mainDirectory, 'Code', 'OUTPUT', 'Variable_Calculation', 'TimeSplitModelData')
        if res == '250m':
            inputDirectory = os.path.join(scratchDirectory, 'OUTPUT', 'Variable_Calculation', 'TimeSplitModelData')
        return inputDirectory

    def GetOutputDirectory(self, mainDirectory, scratchDirectory, res):
        if res == '1km':
            outputDirectory = os.path.join(mainDirectory, 'Code', 'OUTPUT', 'Variable_Calculation', 'CalculateMoreVariables')
            os.makedirs(outputDirectory, exist_ok=True)
        if res == '250m':
            outputDirectory = os.path.join(scratchDirectory, 'OUTPUT', 'Variable_Calculation', 'CalculateMoreVariables')
            os.makedirs(outputDirectory, exist_ok=True)
        return outputDirectory

    def MakeInputDataDirectory(self, inputDirectory, res, t_res, Nz_str):
        inputDataDirectory = os.path.join(inputDirectory, f"{res}_{t_res}_{Nz_str}nz", "ModelData")
        return inputDataDirectory

    def MakeOutputDataDirectory(self, outputDirectory, res, t_res, Nz_str):
        outputDataDirectory = os.path.join(outputDirectory, f"{res}_{t_res}_{Nz_str}nz",self.dataName)
        os.makedirs(outputDataDirectory, exist_ok=True)
        return outputDataDirectory

    def GetTimestepData(self, inputDataDirectory, timeString, VariableName):
        inputDataFile = os.path.join(
            inputDataDirectory,
            f"cm1out_{self.res}_{self.t_res}_{self.Nz_str}nz_{timeString}.h5"
        )
        with h5py.File(inputDataFile, 'r') as f:
            InputData = f[VariableName][:]
        return InputData

    def SaveOutputTimestep(self, outputDataDirectory, res, t_res, timeString, outputDictionary):
        out_file = os.path.join(
            outputDataDirectory,
            f"{self.dataName}_{res}_{t_res}_{self.Nz_str}nz_{timeString}.h5"
        )
        with h5py.File(out_file, 'w') as f:
            for var_name, arr in outputDictionary.items():
                f.create_dataset(var_name, data=arr, dtype=self.dtype, compression="gzip")
        print(f"Saved timestep to output file: {out_file}")

DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str, dataName="Eulerian_Binary_Array", dtype='bool')

In [14]:
####################################
#RUNNING

In [15]:
(ModelData.Ntime) #SET UP JOB ARRAY

133

In [15]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(ModelData.Ntime):
    if np.mod(t,1)==0: print(f'Current time {t}')

    #getting timestring for loading input data
    timeString = ModelData.timeStrings[t]

    #loading input variables
    inputDictionary = GetInputVariables(DataManager.inputDataDirectory, timeString)

    #calculating variables
    outputDictionary = VariableCalculation(inputDictionary)
    
    #outputting
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, ModelData.res, ModelData.t_res, timeString, outputDictionary)
    # break

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Eulerian_Binary_Array/Eulerian_Binary_Array_1km_5min_34nz_0-00-00.h5
Current time 1
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/Eulerian_Binary_Array/Eulerian_Binary_Array_1km_5min_34nz_0-05-00.h5
Current time 2



KeyboardInterrupt


KeyboardInterrupt



In [ ]:
######################################################

In [ ]:
# #READING BACK IN
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'

# with h5py.File(out_file, 'r') as f:
#     A_c = f['A_c'][t]  # Loads the entire A_c array into memory
#     A_g = f['A_g'][t]  # Loads the entire A_g array into memory

In [ ]:
# #####################
# #TESTING
# w_thresh1=0.1
# w_thresh2=0.5
# qcqi_thresh=1e-6
# z=10
# w_t=data['winterp'].isel(time=t,zh=z).data
# qc_t=data['qc'].isel(time=t,zh=z).data
# qi_t=data['qi'].isel(time=t,zh=z).data
# qcqi_t=qc_t+qi_t

# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
# with h5py.File(out_file, 'r') as f:
#     A_c = f['A_c'][t,z]  # Loads the entire A_c array into memory
#     A_g = f['A_g'][t,z]  # Loads the entire A_g array into memory

# cond1=(w_t>=w_thresh1)
# cond2=(qcqi_t<qcqi_thresh)
# print(np.all((cond1&cond2)==A_g))

# cond1=(w_t>=w_thresh2)
# cond2=(qcqi_t>=qcqi_thresh)
# print(np.all((cond1&cond2)==A_c))